In [1]:
### All data but clean
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import math

In [2]:
df = pd.read_csv('data/raw_dataset.csv')
df.shape

(4424, 35)

In [3]:
prev_col = "Previous qualification"

prev_map = {
    15: 3,
    17: 4,
    8: 7,
    9: 7,
    10: 7,
    11: 7,
}

df[prev_col] = df[prev_col].replace(prev_map)

print("Previous qualification value counts after merge:")
print(df[prev_col].value_counts().sort_index())

Previous qualification value counts after merge:
Previous qualification
1     3717
2       23
3      166
4       14
5        1
6       16
7       63
12     162
13       7
14     219
16      36
Name: count, dtype: int64


In [4]:
m_f_qual_cols = ["Mother's qualification", "Father's qualification"]

qual_groups = [
    [1, 7, 8, 10, 11, 12, 14, 15, 17, 19, 20],         # group 1
    [25, 26],                                          # group 2
    [9, 18, 21, 27, 28],                               # group 3
    [6, 13, 16, 22, 23, 29, 31, 32],                   # group 4
    [2, 3, 30],                                        # group 5
    [4, 33],                                           # group 6
    [5, 34],                                           # group 7
    [24],                                              # group 8
]

def build_group_map(groups):
    mapping = {}
    for i, group in enumerate(groups, start=1):
        for code in group:
            mapping[code] = i   # map all codes in that list to group i
    return mapping

qual_map = build_group_map(qual_groups)

for col in m_f_qual_cols:
    df[col] = df[col].replace(qual_map)

print("\nMother's qualification value counts after grouping:")
print(df["Mother's qualification"].value_counts().sort_index())
print("\nFather's qualification value counts after grouping:")
print(df["Father's qualification"].value_counts().sort_index())


Mother's qualification value counts after grouping:
Mother's qualification
1    1263
2      15
3      17
4    2530
5     521
6      49
7      21
8       8
Name: count, dtype: int64

Father's qualification value counts after grouping:
Father's qualification
1    1928
2      10
3    1927
4      32
5     355
6      41
7      19
8     112
Name: count, dtype: int64


In [5]:
nat_col = "Nacionality"
df = df[df[nat_col] == 1].copy()

print("\nShape after keeping only Nationality == 1 (Portugal):", df.shape)
print("Nationality value counts:")
print(df[nat_col].value_counts())


Shape after keeping only Nationality == 1 (Portugal): (4314, 35)
Nationality value counts:
Nacionality
1    4314
Name: count, dtype: int64


In [6]:
m_f_occ_cols = ["Mother's occupation", "Father's occupation"]

occ_groups = [
    [2, 17, 18, 14, 15, 16],                           # group 1
    [3, 19, 20, 21, 22],                               # group 2
    [4, 23, 24, 25, 26],                               # group 3
    [5, 27, 28, 29],                                   # group 4
    [6, 30, 31, 32, 33],                               # group 5
    [7, 34, 35, 43],                                   # group 6
    [8, 36, 37, 38, 39, 40, 41, 42, 44],               # group 7
    [1, 10, 11, 12, 13, 45, 46],                       # group 8
]

occ_map = build_group_map(occ_groups)

for col in m_f_occ_cols:
    df[col] = df[col].replace(occ_map)

print("\nMother's occupation value counts after grouping:")
print(df["Mother's occupation"].value_counts().sort_index())
print("\nFather's occupation value counts after grouping:")
print(df["Father's occupation"].value_counts().sort_index())


Mother's occupation value counts after grouping:
Mother's occupation
1     112
2     325
3     350
4     835
5     541
6      87
7     271
8    1757
9      36
Name: count, dtype: int64

Father's occupation value counts after grouping:
Father's occupation
1     141
2     197
3     382
4     384
5     514
6     244
7     690
8    1452
9     310
Name: count, dtype: int64


In [7]:
first_year_cols = [
    "Curricular units 1st sem (credited)",
    "Curricular units 1st sem (enrolled)",
    "Curricular units 1st sem (evaluations)",
    "Curricular units 1st sem (approved)",
    "Curricular units 1st sem (grade)",
    "Curricular units 1st sem (without evaluations)",
    "Curricular units 2nd sem (credited)",
    "Curricular units 2nd sem (enrolled)",
    "Curricular units 2nd sem (evaluations)",
    "Curricular units 2nd sem (approved)",
    "Curricular units 2nd sem (grade)",
    "Curricular units 2nd sem (without evaluations)"
]
df[first_year_cols] = df[first_year_cols].apply(pd.to_numeric, errors="coerce")
mask_all_zero = (df[first_year_cols] == 0).all(axis=1)

df = df[~mask_all_zero].copy()

print("Shape after removing NAL observations:", df.shape)
print("Removed rows:", mask_all_zero.sum())

Shape after removing NAL observations: (4140, 35)
Removed rows: 174


In [8]:
df.shape

(4140, 35)

In [9]:
target_col = "Curricular units 1st sem (grade)"
semA_enrolled_col = "Curricular units 1st sem (enrolled)"
macro_numeric = ["GDP", "Inflation rate", "Unemployment rate"]

In [10]:
semA_prefix = "Curricular units 1st sem ("
semB_prefix = "Curricular units 2nd sem ("

semA_cols = [c for c in df.columns if c.startswith(semA_prefix)]
semB_cols = [c for c in df.columns if c.startswith(semB_prefix)]

# everything in Sem A except 'enrolled' and the target should be dropped
semA_to_drop = [c for c in semA_cols if c not in {target_col, semA_enrolled_col}]

cols_to_exclude = set(semB_cols + semA_to_drop + ["Target"])
feature_cols = [c for c in df.columns if c not in cols_to_exclude and c != target_col]

print(f"Using {len(feature_cols)} features (Semester B removed, only Sem-A 'enrolled' kept).")

Using 23 features (Semester B removed, only Sem-A 'enrolled' kept).


In [11]:
numeric_cols = [c for c in feature_cols if c in macro_numeric]
categorical_cols = [c for c in feature_cols if c not in numeric_cols]

In [12]:
# enforce dtypes
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
for c in categorical_cols + [semA_enrolled_col]:
    df[c] = df[c].astype("category")

# Target as numeric; drop rows with missing target
y = pd.to_numeric(df[target_col], errors="coerce")
mask = y.notna()
X = df.loc[mask, feature_cols]
y = y.loc[mask]

X.shape


(4140, 23)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [14]:
from sklearn.pipeline import make_pipeline

numeric_pipe = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

categorical_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, numeric_cols),
        ("cat", categorical_pipe, categorical_cols),
    ]
)


In [15]:
lin = LinearRegression()

rf = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_split=3,
    random_state=42,
    n_jobs=-1
)

xgb = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

from sklearn.pipeline import Pipeline
pipe_lin = Pipeline([("prep", preprocess), ("model", lin)])
pipe_rf  = Pipeline([("prep", preprocess), ("model", rf)])
pipe_xgb = Pipeline([("prep", preprocess), ("model", xgb)])

In [16]:
def eval_model(model, name):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    r2  = r2_score(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    rmse = math.sqrt(mean_squared_error(y_test, pred))
    cv_r2 = cross_val_score(model, X_train, y_train, cv=5, scoring="r2").mean()

    print("\n" + "="*70)
    print(name)
    print("="*70)
    print(f"R^2 (test):            {r2:.3f}")
    print(f"CV R^2 (train, 5-fold): {cv_r2:.3f}")
    print(f"MAE:                   {mae:.3f}")
    print(f"RMSE:                  {rmse:.3f}")

for name, mdl in [
    ("Linear Regression", pipe_lin),
    ("Random Forest",     pipe_rf),
    ("XGBoost",           pipe_xgb),
]:
    eval_model(mdl, name)



Linear Regression
R^2 (test):            0.167
CV R^2 (train, 5-fold): 0.184
MAE:                   2.718
RMSE:                  4.099

Random Forest
R^2 (test):            0.152
CV R^2 (train, 5-fold): 0.139
MAE:                   2.673
RMSE:                  4.136

XGBoost
R^2 (test):            0.135
CV R^2 (train, 5-fold): 0.128
MAE:                   2.725
RMSE:                  4.176


In [17]:
preprocess.fit(X_train)
ohe = preprocess.named_transformers_["cat"].named_steps["onehotencoder"] \
      if "onehotencoder" in preprocess.named_transformers_["cat"].named_steps else \
      preprocess.named_transformers_["cat"].named_steps["onehotencoder"[:3]+"ohe"] \
      if "ohe" in preprocess.named_transformers_["cat"].named_steps else \
      preprocess.named_transformers_["cat"].named_steps["onehotencoder"]

In [18]:
cat_names = list(ohe.get_feature_names_out(categorical_cols))
feat_names = numeric_cols + cat_names

Xt = preprocess.transform(X_train)
if hasattr(Xt, "toarray"): Xt = Xt.toarray()
df_design = pd.DataFrame(Xt, columns=feat_names, index=X_train.index)
print("\nTransformed matrix shape:", df_design.shape)
print(df_design.head())


Transformed matrix shape: (3312, 178)
      Unemployment rate  Inflation rate       GDP  Marital status_1  \
4219          -0.300063        0.113566  0.777096               1.0   
4196           0.410068        1.767050 -0.729517               1.0   
192           -1.496074        0.976253  0.155180               1.0   
698            0.297942       -0.533449  0.798994               1.0   
3558          -0.823318       -1.468027 -1.351433               1.0   

      Marital status_2  Marital status_3  Marital status_4  Marital status_5  \
4219               0.0               0.0               0.0               0.0   
4196               0.0               0.0               0.0               0.0   
192                0.0               0.0               0.0               0.0   
698                0.0               0.0               0.0               0.0   
3558               0.0               0.0               0.0               0.0   

      Marital status_6  Application mode_1  ...  \
42